In [7]:
library(SetupCellLayers)
library(Seurat)

Attaching SeuratObject



In [3]:
sobj <- readRDS('../Data/PBMC/pbmc3k.rds')

In [9]:
Idents(sobj) <- 'seurat_annotations'
sobj <- subset(x = sobj, downsample = 100, subset=seurat_annotations!="<NA>")

In [16]:
# Basic Seurat analysis
sobj[["percent.mt"]] <- PercentageFeatureSet(sobj, pattern = "^MT-")
sobj <- subset(sobj, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)
sobj <- NormalizeData(sobj, normalization.method = "LogNormalize", scale.factor = 10000)
sobj <- FindVariableFeatures(sobj, selection.method = "vst", nfeatures = 2000)
all.genes <- rownames(sobj)
sobj <- ScaleData(sobj, features = all.genes)
sobj <- RunPCA(sobj, features = VariableFeatures(object = sobj))
sobj <- JackStraw(sobj, num.replicate = 100)
sobj <- ScoreJackStraw(sobj, dims = 1:20)
sobj <- FindNeighbors(sobj, dims = 1:10)
sobj <- RunUMAP(sobj, dims = 1:10)
# DimPlot(sobj, reduction = "umap")

In [17]:
cl.setup <- compute_modularity(sobj, seq(0.1, 0.5, by=0.1),
                               '../Data/Test-SetupCellLayers/PBMC_modularity.csv')

In [13]:
cl.setup[['sil']] <- compute_silhouette_scores(as.character(cl.setup$mod$'resolution'),
                                               cl.setup$sobj,)

$sobj
An object of class Seurat 
13714 features across 746 samples within 1 assay 
Active assay: RNA (13714 features, 2000 variable features)
 2 dimensional reductions calculated: pca, umap

$mod
  resolution modularity
1    res.0.1     0.9630
2    res.0.2     0.9393
3    res.0.3     0.9222
4    res.0.4     0.9051
5    res.0.5     0.8881
